# Visa Stock Predictive Model
"Refer to Appendix B of the report for instructions on how to obtain prediction values from our model"

### Brief structure of our notebook

#### Creating features:
+ Seasonality removal and detrending
+ Signal processing
+ Raw log returns
+ Combining all the feature

#### Evaluating and selecting features
- Hybrid feature selection

#### Prediction
- Keras neural network

In [165]:
#Install 2 necessary libraries for our model
_ = !pip install keras
_ = !pip install pandas_datareader #This module allows us to read data from yahoo finance
_ = !pip install seasonal

In [166]:
#Import basic data processing and plotting modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data
from pandas.tools.plotting import autocorrelation_plot
from pandas.tools.plotting import scatter_matrix

#Import Keras - A tensorflow wrapper that will allow us to experiment with NNs easily
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import regularizers
from keras.layers import advanced_activations as reLu

#Import modules used to extract seasonality and select features 
import statsmodels.api as sm
from pylab import rcParams
from seasonal import fit_seasons, adjust_seasons
from sklearn.feature_selection import (RFE,RFECV)
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.svm import (SVR,LinearSVR)
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns


## Useful data structures
Below, we define a number of useful variables and data structures that enable our model to be easily adapted to the various stocks we are predicting. By simply making changes to the variables below, all the changes propogate to the entire notebook. This means that we do not have to go through the laborious process of changing multiple variable names when predicting different stocks. The following features were selected due to extensive research that showed they were likely to correlate with the stock due to reasons such as being competitors, clients, suppliers, being in the same industry or an industry that is affected by the industry the stock we are trying to predict is in.

In [167]:
#List out the trackers of the stocks you want to download here (check yahoo finance to get the tracker)
TRACKER_LIST = ["V","MA","AXP","PYPL","BAC","HAWK","BR","PAY","DFS","FDC","JPM","WFC"] #List out the trackers of the stocks you want to download here (check yahoo finance to get the tracker)
NAMES = {
    "V": "Visa Inc.",
  "MA":"Mastercard Incorporated",
  "AXP":"American Express Company",
  "PYPL":"Paypal Holdings Inc",
  "BAC":"Bank of America",
  "HAWK":"Blackhawk Network Holdings Inc",
  "BR":"Broadridge Financial Solution",
  "PAY":"VeriFone Systems Inc",
  "DFS":"Discover Financial Services",
  "FDC":"First Data Corpation",
  "JPM":"JPMorgan Chase & Co.",
  "WFC":"Wells Fargo & Company",   
}

"""
Shift data dictionary to help our algorithm automatically shift stocks in the later part of our code
"shift_data" contains information on whether a particular stock should be shifted or not. 
"0" indicates that it should not be shifted, while "1" indicates that it should be.
"""
shift_data = {
  "V":1,
  "MA":1,
  "AXP":1,
  "PYPL":1,
  "BAC":1,
  "HAWK":1,
  "BR":1,
  "PAY":1,
  "DFS":1,
  "FDC":1,
  "JPM":1,
  "WFC":1,
}

#We define this variable so that we don't need to make multiple changes for each of our models
#ie. we are able to quickly adapt our jupyter notebook to different stocks which we are buying
STOCK_TO_PREDICT = "V"

Although many of the companies here have rather long histories, accuracy seemed to work best when we limited training data to 2013 onwards, possibily due to the constantly evolving payments industry, due to reasons such as new entrants and new technology.

In [ ]:
source = "yahoo"
START = "2013-6-15" #Start date for download
END = "2017-11-17" #End date for download

stocks = data.DataReader(TRACKER_LIST,source,START,END).ix["Close"] #This downloads the data from yahoo and formats it into a dataframe

In [ ]:
stocks.describe()

#### Data Munging

All datafields with NA is filled up with data from previous day.

In [ ]:
stocks = stocks.fillna(method="ffill")
stocks.describe()

In [ ]:
_ = stocks.plot(figsize=(20, 15))

## Seaonality Removal and Detrending

### Why stationarity of data is important
Stationarity of time series data is important because, in its absence, a model describing the data will vary in accuracy at different time points. As such, stationarity is required for sample statistics such as means, variances, and correlations to accurately describe the data at all time points of interest. Therefore, it is crucial for us to transform our dataset such that it is stationary. There are a few ways to achieve this, including:
1. Detrending data
2. Deseasonalizing data

Current literature advances the notion that "if neural networks use deseasonalized time series then the neural networks can focus on learning trend and cyclical components" and thus predict better. Thus, we attempt below to introduce deseasonalized data to our model.

Since the TSA decomposition function allows for both the removal of trend and seasonality, our deseasonalising and detrending steps will be done concurrently.

### Evaluation of the use of deseasonalised data 
The Stats Model's TSA.seasonal_decompose function returns data with the exact same number of data points as the raw data. The additional features can thus be used alongside the daily stock prices. While the additive model is useful when the seasonal variation is relatively constant over time, it returns negative values which makes it hard to apply log returns on them. Thus, after experimentation, we decided to go with the multiplicative model and log-additive model.

As can be seen in the charts below, seasonality on daily data is not very meaningful for both multiplicative and log-additive models. We experimented by adding the deseasonalised features into our model. However, they did not correlate well with the original stock's returns and did little to improve the accuracy of the model.

This can be explained by the charts itself. While there is a slight linear upward trend, the plot of trend data mirrors the observed data. The seasonality charts also deviates from seasonality trends present for the beauty industry where new products are released in anticipation for different seasons. The graphs show that the output did not accurately reflect all real world considerations. This could be explained by the tool used which only made used of moving average to decompose the data.

In [ ]:
deseason = pd.DataFrame()
deseason = stocks.index.copy()
deseason =stocks[STOCK_TO_PREDICT].copy()
deseason = deseason.fillna(deseason.bfill())
rcParams['figure.figsize'] = 7.5, 10

### Mutliplicative deseasonalisation
Below, we experimented with different freq parameters. Since we are unaware of the intrinsic seasonality of the data, we experimented with 4 different freq values:

In [ ]:
decomposition2 = sm.tsa.seasonal_decompose(deseason, model="multiplicative", freq=7) 
decomposition2.plot() 
plt.show()
decomposition3 = sm.tsa.seasonal_decompose(deseason, model="multiplicative", freq=14) 
decomposition3.plot() 
plt.show() 
decomposition4 = sm.tsa.seasonal_decompose(deseason, model="multiplicative", freq=21) 
decomposition4.plot() 
plt.show()
decomposition5 = sm.tsa.seasonal_decompose(deseason, model="multiplicative", freq=28) 
decomposition5.plot() 
plt.show()

#### Log-Additive Model

Log additive model is used as it can convert a multiplicative model into an additive model.
- ln(Xt ) = ln (Tt x St x Rt ) 
- ln(Xt ) = ln(Tt) + ln(St) + ln(Rt)

This can possibly help with the problem of negative values from using the purely additive model. We will add this as additional features. 

In [ ]:
scale = pd.DataFrame()
scale[STOCK_TO_PREDICT] = (stocks[STOCK_TO_PREDICT]/max(stocks[STOCK_TO_PREDICT]))

deseasonLog = pd.DataFrame()
deseasonLog = stocks.index.copy()

log = pd.DataFrame()
log[STOCK_TO_PREDICT] = np.log(scale[STOCK_TO_PREDICT]/scale[STOCK_TO_PREDICT].shift())
deseasonLog =log[STOCK_TO_PREDICT].copy()
deseasonLog = deseasonLog.fillna(deseasonLog.bfill())
rcParams['figure.figsize'] = 7, 10

In [ ]:
decomposition9 = sm.tsa.seasonal_decompose(deseasonLog, model="additive", freq=7) 
decomposition9.plot() 
plt.show()
decomposition10 = sm.tsa.seasonal_decompose(deseasonLog, model="additive", freq=14) 
decomposition10.plot() 
plt.show() 
decomposition11 = sm.tsa.seasonal_decompose(deseasonLog, model="additive", freq=21) 
decomposition11.plot() 
plt.show()
decomposition12 = sm.tsa.seasonal_decompose(deseasonLog, model="additive", freq=28) 
decomposition12.plot() 
plt.show()

## Signal Processing for Stocks

We utilize the robustness of ANN and combine it with signal processing techniques to propose a predictive model for stock market indices.
Signal processing is concerned with processing one and multi-dimensional time series via a spectrum of techniques, filters and algorithms. We are using the Gaussian Zero-Phase Shift filter. 

We are basing our model on the paper titled "STOCK MARKET PREDICTIVE MODEL BASED ON INTEGRATION OF SIGNAL PROCESSING AND ARTIFICIAL NEURAL NETWORK" by Dinesh K. Sharma and Aaron R. Rababaah from University of Maryland. In this paper, log returns is not used, as the filters are sufficient enough to create accurate data that can be used in our NN.

#### Gaussian Zero-Phase Shift Filter (GZ-filter)
Gaussian Zero-Phase Shift filter (GZ-filter) is a very effective digital signal filter. Rababaah (2009) used the GZ-filter to smooth the output of the surveillance situation tracking signal which may be prone to human and random errors. Therefore, we propose the GZ- filter to be used as the smoothing model of the stock market index signal as the first step before we apply further processing such as segmentation, feature vector modeling, training and classification, etc. The concept of GZ-filter is to estimate the signal level at each time instance by computing a kernel of Gaussian-weighted neighborhood of the surrounding data points. 

GZ(S(t), d(t)) = Sum(G(t)S(t)) from t-d(t) to t+d(t)
where S(t): the input noisy signal, t: time, d(t): time interval considered to compute the de-noised signal, G(t): Gaussian probability distribution function.

The uniqueness of the GZ-filter is that unlike conventional signal filters such as moving average or median filter, it retains the phase of the original signal. This means the peaks of the original signal are guaranteed to align with the filtered signal which could be critical if one is interested to identify the exact instances where the signal have abrupt changes.

In [ ]:
from scipy.ndimage import gaussian_filter1d
smooth = pd.DataFrame()

for i in stocks:
  smooth[i] = gaussian_filter1d(stocks[i], 3)

_ = smooth.plot(figsize=(20, 15))

The x-axis is in units of time. As it can be seen, the Gaussian Zero-Phase Shift Filter has smoothened the curve while retaining the phase of the original signal.

#### Signal Normalization

In this step, the data is normalized to the range [0, 1]. This is important since the Artificial Neural Network – Multi-Layered Perceptron (ANN- MLP) model requires normalized input vectors. Mathematically, the normalization is expressed as:

S(i) = S(i)/max(S), where, S(i) is the signal level at index i and S is the entire signal and max is the maximum level of the signal.

In [ ]:
from pandas import concat

scaled_smooth_noDate = pd.DataFrame()
scaled_smooth_data = pd.DataFrame()
intermediate_data = pd.DataFrame()

for column in smooth:
      scaled_smooth_noDate[column] = smooth[column]/max(np.nanmax(smooth[column]))
    
index = stocks.index
scaled_smooth_data = scaled_smooth_noDate.set_index(index)

In [ ]:
print(scaled_smooth_data[STOCK_TO_PREDICT].values)

Plot the new scaled data.

In [ ]:
_ = scaled_smooth_data.plot(figsize=(20,15))

Now we can see the features of the other stocks more clearly.

#### Log Returns

Log returns is used so that our NN model will be only using 2 classes (increase/decrease), as opposed to the paper's original 20 classes of price levels.

In [ ]:
log_returns = pd.DataFrame()

for column in scaled_smooth_data:
  log_returns[column] = np.log(scaled_smooth_data[column]/scaled_smooth_data[column].shift())

In [ ]:
_ = log_returns.plot(figsize=(20,15))

### Test if signal processing makes the data stationary

Now to test if the data is stationary we used the Augmented Dickey Fuller Test. From the test data, we can be assured that the data set is now stationary as the p value is extremely low (below 5% or 1%), this means that the null hypothesis of the process having no unit root can be rejected. This means that the time series has no time-dependent structure. Therefore we can safely use this method to make our time series stationary. Thus, even without using detrending or deseasonalizing the data, signal processing can be used in place of detrending and deseasonalizing.

In [ ]:
from statsmodels.tsa.stattools import adfuller
resid = log_returns[STOCK_TO_PREDICT].values

#remove empty values which will throw problems during the test
x = resid[~np.isnan(resid)] 

result = adfuller(x)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.4f' % (key, value))

Now let's look at autocorrelations

In [ ]:
from pandas.tools.plotting import autocorrelation_plot

fig_2 = plt.figure()
fig_2.set_figwidth(20)
fig_2.set_figheight(15)

for column in log_returns:
  autocorrelation_plot(log_returns[column])

### Finalise signal processed data
Below, we take the final step of shifting our signal processed features to prepare them for our neural network model

In [ ]:
signal_processed_shifted = pd.DataFrame()

#We then shift all the data according to their respective time zones
for stock in log_returns:
  if shift_data[stock] == 1:
    signal_processed_shifted[stock] = log_returns[stock].shift() #Shift stocks in the same time zone as the stock to be predicted by 1 day
  else:
    signal_processed_shifted[stock] = log_returns[stock] #Don't shift the other stocks

### Add raw log returns (without signal processing smoothening)
We also add the "vanilla" raw log returns of each stock to create additional features for our NN model

In [ ]:
raw_log_returns = pd.DataFrame()

for stock in stocks:
  raw_log_returns[stock + "_raw"] = np.log(stocks[stock]/stocks[stock].shift())
  
for stock in raw_log_returns:
  if shift_data[stock[:-4]] == 1:
    raw_log_returns[stock] = raw_log_returns[stock].shift() #Shift stocks in the same time zone as the stock to be predicted by  1 day

### Combine both signal processed and raw log return data
We combine the two dataframes which were computed above to obtain our final feature set for our neural network model

In [ ]:
shifted_returns = pd.merge(raw_log_returns, signal_processed_shifted, left_index=True, right_index=True, how='inner')

Drop missing data

In [ ]:
shifted_returns = shifted_returns.dropna()

### Add classification labels (ie. whether log returns are positive or negative)

Below, we obtain the classification label for our stock separately and add it into our dataframe. This label is what our neural network model will be trained on

In [ ]:
stock_to_predict_log_returns = pd.DataFrame()
stock_to_predict_log_returns["temp_stock_to_predict"] = np.log(stocks[STOCK_TO_PREDICT]/stocks[STOCK_TO_PREDICT].shift())

#Obtain the labels
stock_to_predict_log_returns["STOCK_RISES"] = 0
stock_to_predict_log_returns.ix[stock_to_predict_log_returns["temp_stock_to_predict"] >=0, "STOCK_RISES"] = 1
stock_to_predict_log_returns = stock_to_predict_log_returns.drop("temp_stock_to_predict",axis=1)

#Add label column into the shifted_returns dataframe
shifted_returns = pd.merge(shifted_returns, stock_to_predict_log_returns, left_index=True, right_index=True, how='inner')

Show the correlations between the features and the stock to be predicted

In [ ]:

#shifted_returns.corr()[STOCK_TO_PREDICT + "_raw"] #We drop the label column here because it's irrelevant to determining if the features are correlated with the stock

In [ ]:
shifted_returns.describe()

## Hybrid Feature Selection

Aside from using the default correlation function, there are many are ways to help us determine which features are most important. Including too many features can actually worsen accuracy as there is less can mean more noise and thus less pattern for the neural network to learn from.As such, we use an Hybrid Feature Selection strategy similar to the one covered in the reading.We record the score of all the features in terms of how useful it was to predict our target variable from a variety of different methods. The mean score then used to determine the overall usefulness of the feature.

In [ ]:
Y = shifted_returns[STOCK_TO_PREDICT + "_raw"].values
X3 = shifted_returns.drop(['STOCK_RISES'],axis=1)
X2 = X3.drop([STOCK_TO_PREDICT + "_raw"],axis=1)
X = X3.as_matrix()
colnames = X2.columns
# Define dictionary to store our rankings
ranks = {}
# Create our function which stores the feature rankings to the ranks dictionary
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

Random Forest was indicated as the most useful classifier in the paper, and we have included that. However, several of the feature selectors discussed in the paper do not seem that useful after our own experimentation, such as Bayesian methods, and as such we have selected some others of our own.

Linear Regression is one such method. We also implemented this a second way by using it in conjunction with a tool in Sklearn known as Recursive Feature Elimination or RFE.RFE uses a model such as Linear Regression to select either the best or worst-performing feature, and then excludes this feature. The whole process is then iterated until all features in the dataset are used up.Support Vector Regression, which is characterized by usage of kernels, absence of local minima, sparseness of the solution and capacity control obtained by acting on the margin, or on number of support vectors, is also used with RFE. 

Ridge regression,which performs ‘L2 regularization‘, i.e. it adds a factor of sum of squares of coefficients in the optimization objective, is also used, along with the default correlation function.


In [ ]:
# Using Random Forest
rf = RandomForestRegressor(n_jobs=-1, n_estimators=50, verbose=3)
rf.fit(X,Y)
ranks["RF"] = ranking(rf.feature_importances_, colnames)

#Run Default Correlation Function
ranks["Corr"] = shifted_returns.corr()[STOCK_TO_PREDICT + "_raw"]
#for i in colnames:
#    ranks["Corr"][i] = round(ranks["Corr"][i], 4)

# Construct our Linear Regression model and rank features using Recursive Feature Elimination
lr = LinearRegression(normalize=True)
lr.fit(X,Y)
#stop the search when only the last feature is left
rfe = RFE(lr, n_features_to_select=1, verbose =3 )
rfe.fit(X,Y)
ranks["RFE"] = ranking(list(map(float, rfe.ranking_)), colnames, order=-1)

#rfe2 using Support Vector Regression and RFE
svr = LinearSVR()
rfe2 = RFE(estimator=svr, n_features_to_select=1, step=1)
rfe2.fit(X, Y )
ranks["RFE2"] = ranking(list(map(float, rfe2.ranking_)), colnames, order=-1)

# Using Linear Regression
lr = LinearRegression(normalize=True)
lr.fit(X,Y)
ranks["LinReg"] = ranking(np.abs(lr.coef_), colnames)

# Using Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X,Y)
ranks['Ridge'] = ranking(np.abs(ridge.coef_), colnames)

# Create empty dictionary to store the mean value calculated from all the scores
r = {}
for name in colnames:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)

methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("\t%s" % "\t".join(methods))
for name in colnames:
    print("%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods]))))
# Put the mean scores into a Pandas dataframe
meanplot = pd.DataFrame(list(r.items()), columns= ['Feature','Mean Ranking'])

# Sort the dataframe
meanplot = meanplot.sort_values('Mean Ranking', ascending=False)

# Let's plot the ranking of the features
sns.factorplot(x="Mean Ranking", y="Feature", data = meanplot, kind="bar", 
               size=10, aspect=1.9, palette='coolwarm')

Based on the above information, we then decide how many features we want to keep by setting a minimum threshold for the mean score of each feature, anything lower than this figure will be removed by the function below. Based on experimentation 0.2 seems to allow the optimal number of features that do help to improve the metrics like accuracy subsequently.

In [ ]:
numfeatures = len(colnames)
print(ranks["Mean"])
def dropfeatures(data,colnames,threshold):
    for i in colnames:
        if ranks["Mean"][i] <= threshold:
            print(i+" dropped")
            data = data.drop([i],axis=1)
    return data
shifted_returns = dropfeatures(shifted_returns,colnames,0.2)
shifted_returns.describe()

## Neural Network with Keras

We now create our labels dataframe, which contains the labels (whether the stock rises or falls) for both training and testing data

In [ ]:
labels = pd.DataFrame()
labels = shifted_returns.loc[:,["STOCK_RISES"]] #Obtain labels of the data

Below, we define some of the key variables we are going to use to split our data into 3 parts:
- Number of days of testing data that we want to use
- Number of days of training data that we want to use
- Number of days to predict (typically 1 day, ie. the next day)

In [ ]:
#Change these parameters according to the number of days of testing data you want to use and the number of features you're placing into keras

DAYS_TO_PREDICT = 1
LENGTH_TESTING = int(0.25*len(labels))
LENGTH_TRAINING = len(labels) - LENGTH_TESTING
NUM_OF_FEATURES = len(shifted_returns.columns)-1

In [ ]:
x = shifted_returns.drop("STOCK_RISES",axis=1).values
y = labels.values.reshape((len(labels), 1)) #Change pandas dataframe to a numpy array for use with Keras

TRAINING_DATA_DAYS = -LENGTH_TESTING - DAYS_TO_PREDICT
x_train = x[:TRAINING_DATA_DAYS]
y_train = y[:TRAINING_DATA_DAYS]

x_test = x[TRAINING_DATA_DAYS : -DAYS_TO_PREDICT]
y_test = y[TRAINING_DATA_DAYS : -DAYS_TO_PREDICT]

x_to_predict = x[-DAYS_TO_PREDICT]

Here, two metric functions is defined as required by Keras: precision and recall

In [ ]:
import keras.backend as K

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

Next, we define our model. The current model uses a four layer NN with 20,50,20 and 50 neurons respectively.Increasing the number beyond these layers or neurons seem to cause potential overfitting til to the lower resulting accuracy. Usage of adam has also been more effective when coupled with feature selection for more significant improvements.Regularisation has had marginal benefit, especially using l2 instead of l1, and the current degree of regularisation seems to work best for this model as well.

In [ ]:
model = Sequential()

#LeakyRelu tried, no diff in results
model.add(Dense(20, activation = reLu.LeakyReLU(alpha=0.3), input_dim=NUM_OF_FEATURES, activity_regularizer=regularizers.l2(0.01)))
model.add(Dense(50, activation = reLu.LeakyReLU(alpha=0.3),activity_regularizer=regularizers.l2(0.01)))
model.add(Dense(20, activation = reLu.LeakyReLU(alpha=0.3),activity_regularizer=regularizers.l2(0.01)))
model.add(Dense(50, activation = reLu.LeakyReLU(alpha=0.3),activity_regularizer=regularizers.l2(0.01)))

model.add(Dense(1,activation = "sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy',precision, recall])

We now train the neural model using the training data defined above. Epochs ranging from 50 to 250 were tested, as well as batch sizes from 5 to 30 before deciding on the current number.

In [ ]:
model.fit(x_train, y_train,
          epochs=200,
          batch_size=20, verbose = 1)

Do a quick check to see that our model has a reasonable level of accuracy, precision and recall

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=15)

We now output the vital test results from our model:

In [ ]:
print(model.metrics_names)
print(score)

We can finally use the model to make a prediction:

In [ ]:
x_to_predict = x_to_predict.reshape(1,NUM_OF_FEATURES) #Reshape the data so that it fits the input structure of our model
result = model.predict(x_to_predict)

print("Probability that stock will rise in the next day: " + str(result[0][0]))